# Flattening the curve in Brazil 
> Active cases and estimated number of ICUs available for covid-19 patients

- toc: true
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [jupyter]
- image: images/brazil-flatten-the-curve.png

In [1]:
#hide
import pandas as pd
import altair as alt
import math
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

/usr/local/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#hide
STATE_COLUMN = "State"
DATE_COLUMN = "Date"
CONFIRMED_CASES = 'Confirmed Cases'
DEATHS = 'Deaths'
ACTIVE_COLUMN = "Active Cases"
VALUE = "Value"
CASES_NEED_ICU = 0.12
POPULATION = 211000000
ICU_PER_100k = 20
TOTAL_ICU = (211000000 / 100000 * 20) + 1456
AVAILABLE_PERCENTAGE_ICU = 0.3
TOTAL_AVAILABLE_ICU = TOTAL_ICU * AVAILABLE_PERCENTAGE_ICU
CASES_NEED_ICU_COLUMN = "Need ICU"
CASES_DONT_NEED_ICU_COLUMN = "Do not need ICU"
PATIENTS = "Patients"

In [3]:
#hide
url_cases = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
url_deaths = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
url_recoveries = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

src = pd.read_csv(url_cases)
df_cases = src[(src['Country/Region'] == 'Brazil')]
df_cases['base'] = 'cases'

src = pd.read_csv(url_recoveries)
df_recoveries = src[(src['Country/Region'] == 'Brazil')]
df_recoveries['base'] = 'recoveries'

src = pd.read_csv(url_deaths)
df_deaths = src[(src['Country/Region'] == 'Brazil')]
df_deaths['base'] = 'deaths'

# Join databases
df = [df_cases, df_recoveries, df_deaths]
df = pd.concat(df)

# Drop unused columns
df = df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1)

# Add active cases row
df = df.set_index('base')
new_row = df.loc['cases'] - df.loc['recoveries']
new_row.name = 'actives'
df = df.append([new_row])
# Reset index after added new row
df = df.reset_index()
df = df.rename(columns={"index":"base"})

# Melt structure (unpivot)
dt_cols = list(df.columns[~df.columns.isin(['base','index'])])
df = df.melt(id_vars=['base'], value_vars=dt_cols)
df = df.rename(columns={
     "base": STATE_COLUMN, 
     "variable":DATE_COLUMN, 
     "value":VALUE,
     "actives": ACTIVE_COLUMN
})
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')
df = df.sort_values(by=[DATE_COLUMN])

# Final database has only active cases
df = df[df[STATE_COLUMN].isin(['actives'])]

data = df.copy()

<ipython-input-3-26cb67d50e93>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cases['base'] = 'cases'
<ipython-input-3-26cb67d50e93>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recoveries['base'] = 'recoveries'
<ipython-input-3-26cb67d50e93>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [4]:
#hide
df = data.copy()
STATE_COLUMN = PATIENTS
df[VALUE] = df[VALUE] * (CASES_NEED_ICU)
df[STATE_COLUMN] = CASES_NEED_ICU_COLUMN
df = df.rename(columns={VALUE:ACTIVE_COLUMN})

In [5]:
#hide_input
input_dropdown = alt.binding_select(options=df[STATE_COLUMN].unique())
selection = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(ACTIVE_COLUMN, axis=alt.Axis(title=ACTIVE_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, ACTIVE_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title=[
        "Flatten the curve - only active cases that should need ICU",
        "*see assumptions"
    ]
).add_selection(
    selection
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("ICUs for COVID19 patients: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU))
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 | legend

alt.HConcatChart(...)

In [6]:
#hide
df1 = data.copy()
STATE_COLUMN = PATIENTS
df1[VALUE] = df1[VALUE] * (CASES_NEED_ICU)
df1[STATE_COLUMN] = CASES_NEED_ICU_COLUMN
df2 = data.copy()
df2[VALUE] = df2[VALUE] * (1-CASES_NEED_ICU)
df2[STATE_COLUMN] = CASES_DONT_NEED_ICU_COLUMN
df = pd.concat([df1, df2])
df = df.rename(columns={VALUE:ACTIVE_COLUMN})

In [7]:
#hide_input
input_dropdown = alt.binding_select(options=df[STATE_COLUMN].unique())
selection = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(ACTIVE_COLUMN, axis=alt.Axis(title=ACTIVE_COLUMN)),
    color=color,
    tooltip=[DATE_COLUMN, ACTIVE_COLUMN],
    order=alt.Order(
    STATE_COLUMN,
    sort='descending'
    )
).properties(
    title=[
        "Flatten the curve - all active cases",
        "*see assumptions"
    ]
).add_selection(
    selection
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("ICUs for COVID19 patients: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU))
)

x2line = alt.Chart(pd.DataFrame({'y': [TOTAL_ICU]})).mark_rule(color='darkred', strokeWidth=2).encode(
    y='y:Q'
)
text2 = x2line.mark_text(align='left', x=5, dy=10, color='darkred', strokeWidth=1).encode(
    text=alt.value("Total of ICUs: "+"{:.0f}".format(TOTAL_ICU))
)

chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 + x2line + text2 | legend

alt.HConcatChart(...)

In [8]:
#hide_input
print("Assumptions:")
print("- Brazilian population is estimated in "+"{:.0f}".format(POPULATION/1000000)+" millions inhabitants (source: ibge)")
print("- Brazil has around "+str(ICU_PER_100k)+" ICUs for 100 thousand inhabitants (source: bcc). New ICU to fight covid-19 (source: Ministry of Health)")
print("- Based on data above, it is estimated "+"{:.0f}".format(TOTAL_ICU)+" ICUs for the whole country.")
print("- {:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% of the ICUs would be available for covid-19 patients (source: oglobo)")
print("- Around "+"{:.0f}".format(CASES_NEED_ICU*100)+"% of covid-19 patients need ICU (source: the lancet paper)")

Assumptions:
- Brazilian population is estimated in 211 millions inhabitants (source: ibge)
- Brazil has around 20 ICUs for 100 thousand inhabitants (source: bcc). New ICU to fight covid-19 (source: Ministry of Health)
- Based on data above, it is estimated 43656 ICUs for the whole country.
- 30% of the ICUs would be available for covid-19 patients (source: oglobo)
- Around 12% of covid-19 patients need ICU (source: the lancet paper)


Developed by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). Based on the work of [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html). 

Data sources:
- covid19: [brasil.io](https://brasil.io/home/)
- estimated population: [ibge](https://www.ibge.gov.br/apps/populacao/projecao//)
- number of ICU: [bcc](https://www.bbc.com/portuguese/brasil-52137553) e [ministério da saúde](https://www.saude.gov.br/noticias/agencia-saude/46772-brasil-ganha-reforco-de-1-134-leitos-de-uti-no-combate-ao-coronavirus)
- available ICUs: [oglobo](https://oglobo.globo.com/sociedade/coronavirus/coronavirus-ministerio-estima-que-sus-tem-de-12-13-mil-leitos-de-uti-disponiveis-para-atender-pacientes-1-24328523)
- patients that need ICU: [the lancet paper](https://linkinghub.elsevier.com/retrieve/pii/S2213260020301612)